In [193]:
import gensim 
from gensim.models import Word2Vec
model = Word2Vec.load("word2vec.model")

In [194]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
#import matplotlib.pyplot as plt
#%matplotlib inline

import warnings;
warnings.filterwarnings('ignore')

In [730]:
df = pd.read_excel('mist_dataset.xlsx', header=None)
df.head()

,0,1
0,MIST,Military Institute of Science and Technology i...
1,location of MIST,Military Institute of Science and Technology(M...
2,Is there military and civil students both,"Though a military institution, both military a..."
3,dress code in MIST,Civil students are to wear dress with displaye...
4,Admission procedure,"First, students are screened based on their gr..."


In [731]:
df.dropna(inplace=True)

In [732]:
from nltk.corpus import stopwords
import re
def compute_vec(query):
    sm = np.zeros(150)
    for tok in query.split(" "):
        sm = sm + model.wv[tok]
    return sm

In [733]:
import numpy as np

def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

In [734]:
import nltk
#nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
nltk.download('punkt')

def clean1(text1):
  #punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  #no_punct = ""
  #for char in text1:
   #  if char not in punctuations:
   #      no_punct = no_punct + char
  exclist = string.punctuation
  # remove punctuations and digits from oldtext
  table_ = str.maketrans('', '', exclist)
  #newtext = oldtext.translate(table_)
  text = ' '.join(text1.translate(table_).split())
  #sents = ['i like cars', 'cats are the best']
  word_list = nltk.word_tokenize(text)
  #print(word_list)
  # Lemmatize list of words and join
  lemmatizer = WordNetLemmatizer()
  lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
  #print(lemmatized_output)
  text=lemmatized_output
  #text = no_punct
  text = re.sub(r'\[[0-9]*\]',' ',text)
  text = re.sub(r'\s+',' ',text)
  text = text.lower()
  text = re.sub(r'\d',' ',text)
  text = re.sub(r'\s+',' ',text)
  stopwords = ['i','how', 'you', 'your','me','must', 'an', 'a', 'at','any','is','the','of','some','in','want','need','to','can','could','please','about','tell','that','are','have','what','who','there','any', 'do', 'does']
  querywords = text.split()

  text  = [word for word in querywords if word not in stopwords]
  text = ' '.join(text)
  #stop_words = set(stopwords.words('english'))
  #no_stop = ""
  #for char in text:
   #  if char not in stop_words:
    #     no_stop = no_stop + char
  #text = no_stop  
  #print(text)
  return text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\FARIHA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\FARIHA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [735]:
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
import re
def clean(para):
  #sentences = nltk.sent_tokenize(para)
  sentences =  clean1(para)
  sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
  for i in range(len(sentences)):
    sentences[i] = [word for word in sentences[i] if word not in stopwords.words('english')]

In [736]:

import numpy as np
def get_response(query):
  min=0.4
  flag=0
  ques=df[0]
#query = "where is mist situated"
  #print('ques : ')
  query=clean1(query)
  q = compute_vec(query)
  for i in range(len(ques)):
      sn=clean1(ques[i])
      sn=ques[i].lower()
      l=compute_vec(sn)
      l1=cos_sim(q, l)
      if min < l1:
        min=l1
        flag=1
        ans=i
  if flag == 0:
      return 'sorry!we dont have that information'
  else:
    
      return df[1][ans]
    

In [737]:
#query = input()
#get_response(query)

In [738]:
import tkinter as tk
from tkinter import *



def ret(event):
    
    
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    
    if msg != 'sorry! we dont have that information':
        ChatBox.config(state=NORMAL)
        ChatBox.insert(END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="#070B19", font=("Verdana", 12 ))
    
        
        res = get_response(msg)
        
        ChatBox.insert(END, "Help Desk: " + res + '\n\n')
    else:
        response="Sorry! we dont have that information"
        ChatBox.insert(END, "Mist Help Desk: " + response + '\n\n')
            
     
  
    ChatBox.config(state=DISABLED)
    ChatBox.yview(END)
 

In [742]:
root = tk.Tk()
root.title("Mist Help Desk")

root.geometry("400x500")
root.resizable(width=FALSE, height=FALSE)

#Create Chat window

ChatBox = Text(root, bd=0, bg="#CEF6CE", height="8", width="50", font="Arial",)

ChatBox.config(state=NORMAL)
ChatBox.config(foreground="#070B18", font=("Verdana", 12 ),)
ChatBox.insert(END, "Welcome to MIST Help Desk! how can I help you?" + '\n\n')
#Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set

SendButton = tk.Button(root, font=("Verdana",12,'bold'), text="Send", width="10", height=5,
                    bd=0, bg="#21610B", activebackground="#3c9d9b",fg='white',
                    command= send )


root.bind('<Return>', ret)
SendButton.pack()

#Create the box to enter message
EntryBox = Text(root, bd=0, bg="white",width="33", height="5", font="Arial",)
#EntryBox.bind("<Return>",send)
scrollbar.place(x=376,y=6, height=386)
ChatBox.place(x=6,y=6, height=386, width=370)

EntryBox.place(x=128, y=401, height=90, width=269)

SendButton.place(x=6, y=401, height=90)

root.mainloop()